<a href="https://colab.research.google.com/github/tyleralc/DSCI_550_Data_Geeks/blob/main/Problem4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from googlesearch import search
import time
import math

In [ ]:

# PLOS
def plos_one(journal_id):
  if 'pone' in journal_id:
    link = 'https://journals.plos.org/plosone/article?id=' + journal_id
  if 'pbio' in journal_id:
    link = 'https://journals.plos.org/plosbiology/article?id=' + journal_id
  if 'pgen' in journal_id:
    link = 'https://journals.plos.org/plosgenetics/article?id=' + journal_id
  if 'ppat' in journal_id:
    link = 'https://journals.plos.org/plospathogens/article?id=' + journal_id
  if 'pntd' in journal_id:
    link = 'https://journals.plos.org/plosntds/article?id=' + journal_id
  content = requests.get(link)
  soup = BeautifulSoup(content.content, 'html.parser')
  # author_names = list()
  affiliate_universities = list()
  #for link in soup.find_all('meta', {'name': 'citation_author'}):
  #  author_names.append(link['content'])
  for link in soup.find_all('meta', {'name': 'citation_author_institution'}):
    affiliate_universities.append(link['content'])
  return affiliate_universities

#MBIO
def mBio(journal_id):
  link = 'https://journals.asm.org/doi/' + journal_id
  content = requests.get(link)
  soup = BeautifulSoup(content.content, 'html.parser')
  print(soup)
  affiliate_universities = list()
#  for link in soup.find_all('meta', {'name': 'citation_author_institution'}):
#    affiliate_universities.append(link['content'])
#  return affiliate_universities

affiliate_university = list()

for i in range(48):
  print(i)
  all_affiliate_unversities = plos_one(journal_link[i])
  affiliate_university.append(all_affiliate_unversities[0])

print(affiliate_university)
print(len(affiliate_university))


In [ ]:
#dataframe
# encoding = "ISO-8859-1"
df = pd.read_csv("Bik dataset - papers with endpoint reached.tsv", sep='\t',encoding = 'utf-8')
journal_link = df['DOI']
all_authors = df['Authors'].tolist()
print(len(all_authors))


214


In [ ]:
#Pubmed (publication rate, duration of career, other journals publisehd)
#output total publications
def pubmed_results(url):
  pub = ''
  content = requests.get(url)
  soup = BeautifulSoup(content.content, 'html.parser')
  #print(soup)
  for link in soup.find_all('meta', {'name': 'log_resultcount'}):
    pub = link['content']
  return pub

#output years paper were published
def pubmed_year(url):
  year = []
  content = requests.get(url)
  soup = BeautifulSoup(content.content, 'html.parser')
  #print(soup)
  for link in soup.find_all('span', {'class': 'docsum-journal-citation short-journal-citation'}):
    year.append(link.get_text()[-5:-1])
  return year

duration_career = []
pub_rate = []
i = 0
for i in range(5):
  #pub_total = []
  #pub = []
  temp_authors = all_authors[i].split(', ')
  dur = []
  rate = []
  for j in range(len(temp_authors)):
    pub_year = []
    temp_journals = []
    link = 'https://pubmed.ncbi.nlm.nih.gov/?term='
    for k in range(len(temp_authors[j].split(' '))):
      link = link + temp_authors[j].split(' ')[k] + '%20'
    link = link[:-3]
    pub_year = pubmed_year(link)
    pub_num = pubmed_results(link)
    #pub.append(pub_num)
    if pub_num == '':
      pub_num = 1
    if int(pub_num) > 200:
      pub_num = 200
    for k in range(1,math.ceil(float(pub_num)/10)):
      page_link = link + '&page=' + str(k+1)
      pub_year = pub_year + pubmed_year(page_link)
    #dur.append(str(int(max(pub_year)) - int(min(pub_year))))
    if j == 0:
      dur = int(max(pub_year)) - int(min(pub_year))
    temp_rate = round(float(pub_num)/(float(max(pub_year)) - float(min(pub_year))),2)
    rate.append(temp_rate)
  #pub_total.append(pub)
  duration_career.append(dur)
  pub_rate.append(rate)
  #print(pub_total)
  print(i)
  print(duration_career)
  print(pub_rate)


0
[3]
[[1.0, 1.0, 0.95]]
1
[3, 11]
[[1.0, 1.0, 0.95], [0.45, 0.28, 1.0, 3.2, 1.1, 0.91, 3.9]]
2
[3, 11, 14]
[[1.0, 1.0, 0.95], [0.45, 0.28, 1.0, 3.2, 1.1, 0.91, 3.9], [1.14, 11.76, 5.42, 8.33, 1.15, 0.94]]
3
[3, 11, 14, 19]
[[1.0, 1.0, 0.95], [0.45, 0.28, 1.0, 3.2, 1.1, 0.91, 3.9], [1.14, 11.76, 5.42, 8.33, 1.15, 0.94], [0.79, 1.07, 1.27, 1.67]]
4
[3, 11, 14, 19, 19]
[[1.0, 1.0, 0.95], [0.45, 0.28, 1.0, 3.2, 1.1, 0.91, 3.9], [1.14, 11.76, 5.42, 8.33, 1.15, 0.94], [0.79, 1.07, 1.27, 1.67], [2.05, 18.18, 0.06, 33.33, 11.11, 0.8, 3.89]]


In [ ]:
#get first 3 unique journals from first page
def pubmed_journals(url):
  journals = []
  content = requests.get(url)
  soup = BeautifulSoup(content.content, 'html.parser')
  for link in soup.find_all('span', {'class': 'docsum-journal-citation short-journal-citation'}):
    entry = link.get_text().split('.')[0]
    if entry not in journals:
      journals.append(entry)
  return journals[0:3]


other_journals = []
for i in range(len(all_authors)):
  temp_authors = all_authors[i].split(', ')
  pub_journals = []
  for j in range(len(temp_authors)):
    temp_journals = []
    link = 'https://pubmed.ncbi.nlm.nih.gov/?term='
    for k in range(len(temp_authors[j].split(' '))):
      link = link + temp_authors[j].split(' ')[k] + '%20'
    link = link[:-3]
    temp_journals = pubmed_journals(link)
    pub_journals.append(temp_journals)
  other_journals.append(pub_journals)
  print(i)
df['Other Journals'] = other_journals
print(df)

In [ ]:
# Creating Excel Writer Object from Pandas  
df.to_csv('test.tsv', sep='\t')   

# New Section

In [ ]:
#Research Gate 
def research_gate(url):
  content = requests.get(url)
  soup = BeautifulSoup(content.content, 'html.parser')
  #print(soup)
  for link in soup.find_all('h2', {'class': 'nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit'}):
    txt = link.get_text()
    if 'Publications' in txt:
      txt = txt.replace('Publications (', '')
      publication_num = txt.replace(')','')
      return publication_num

def research_gate_profile(url):
  content = requests.get(url)
  soup = BeautifulSoup(content.content, 'html.parser')
  #print(soup)
  for link in soup.find_all('div', {'data-testid': 'publicProfileStatsPublications'}):
    publication_num = link.get_text()
    return publication_num

for i in range(20):
  temp_authors = all_authors[i].split(', ')
  pub = ''
  for j in range(len(temp_authors)):
    link = ''
    query = temp_authors[j] + ' Research Gate'
    time.sleep(4)
    for k in search(query, stop=5, pause=2):
      if 'https://www.researchgate.net/scientific-contributions/' in k:
        link = k
        print(link)
        pub = pub + research_gate(link) + ', '
        print(pub)
        break
      elif 'https://www.researchgate.net/profile/' in k:
        link = k
        pub = pub + research_gate_profile(link) + ', '
        print(pub)
        break
  pub = pub[:-2]
  publication_rate.append(pub)
print(publication_rate)

https://www.researchgate.net/scientific-contributions/Inka-Regine-Weingaertner-2004447803


TypeError: ignored

In [ ]:
#Hindawi
link = 'https://www.hindawi.com/journals/'+journal_id+'/'
content = requests.get(link)
soup = BeautifulSoup(content.content, 'html.parser')

In [ ]:
#PMID
journal_id = journal_link[52].replace('PMID: ', '')
link = 'https://pubmed.ncbi.nlm.nih.gov/' + journal_id + '/'
content = requests.get(link)
soup = BeautifulSoup(content.content, 'html.parser')

main=soup('main')
for tag in main:
  divs=tag.find_all('affiliation-link')
print(divs)

#also has link of form 10.1111/j.1365-2672.2010.04795.x. ==> https://pubmed.ncbi.nlm.nih.gov/20636343/ (how do we work with this??? there is a DOI on the website)
#10.1016/S0169-5002(01)00212-4
#10.1016/j.lungcan.2006.06.001
#10.1016/j.jaut.2005.09.016 ==> https://pubmed.ncbi.nlm.nih.gov/16271292/
#10.1016/j.cyto.2003.11.014
#10.1126/science.1156725
#10.1038/34214
#pubmed has most of the articles it looks like by DOI
#10.1038/nature07034
#10.1038/onc.2013.184
#10.3892/ijo.2012.1714
#10.1016/j.ccr.2013.04.019
#10.1016/j.ccell.2014.09.008


NameError: ignored

In [ ]:
#Wiley online library
link = 'https://onlinelibrary.wiley.com/doi/' + journal_id 
content = requests.get(link)
soup = BeautifulSoup(content.content, 'html.parser')

In [ ]:
#BMC micro biology
link = 'https://bmcmicrobiol.biomedcentral.com/articles/' + journal_id
content = requests.get(link)
soup = BeautifulSoup(content.content, 'html.parser')

In [ ]:
#BMC
link = 'https://genomebiology.biomedcentral.com/articles/' + journal_id
content = requests.get(link)
soup = BeautifulSoup(content.content, 'html.parser')

#also BMC ==> 10.1186/bcr3128

In [ ]:
#10.1083/jcb.201201133
#cant find a good link for this one

In [ ]:
#PNAS
#10.1073/pnas.1101273108

In [ ]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://pubmed.ncbi.nlm.nih.gov/?term=LPS%20Unmasking%20of%20Shigella%20flexneri%20Reveals%20Preferential%20Localisation%20of%20Tagged%20Outer%20Membrane%20Protease%20IcsP%20to%20Septa%20and%20New%20Poles')
soup = BeautifulSoup(response.content, 'html.parser')
soup.find_all('li', {"data-affiliation-id": "affiliation-1"})

for text in soup.find_all('li', {"data-affiliation-id": "affiliation-1"}):
  print(text.text)

1 Discipline of Microbiology and Immunology, School of Molecular and Biomedical Science, University of Adelaide, Adelaide, Australia.
1 Discipline of Microbiology and Immunology, School of Molecular and Biomedical Science, University of Adelaide, Adelaide, Australia.


In [ ]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://pubmed.ncbi.nlm.nih.gov/?term=LPS%20Unmasking%20of%20Shigella%20flexneri%20Reveals%20Preferential%20Localisation%20of%20Tagged%20Outer%20Membrane%20Protease%20IcsP%20to%20Septa%20and%20New%20Poles')
#put doi after term=
soup = BeautifulSoup(response.content, 'html.parser')
soup.find_all('li', {"data-affiliation-id": "affiliation-1"})

for text in soup.find_all('li', {"data-affiliation-id": "affiliation-1"}):
  print(text.text[2:])
  break

Discipline of Microbiology and Immunology, School of Molecular and Biomedical Science, University of Adelaide, Adelaide, Australia.


In [ ]:
# Use the article DOI to search pubmed directly

In [ ]:
response=requests.get('https://orcid.org/orcid-search/search?searchQuery=keith%20burghardt')
print(response.text)

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8"/>
    <title>ORCID</title>
    <base href="/"/>
    <meta name="robots" content="all"/>

    <meta name="viewport" content="width=device-width, initial-scale=1"/>
    <script src="/cdn-cgi/apps/head/N3f49hMwWzdVXaiTaxh9CqyeUQg.js"></script><link rel="icon" type="image/x-icon" href="./assets/icons/favicon.ico"/>
    <style type="text/css">@font-face{font-family:'Material Icons';font-style:normal;font-weight:400;src:url(https://fonts.gstatic.com/s/materialicons/v125/flUhRq6tzZclQEJ-Vdg-IuiaDsNa.woff) format('woff');}.material-icons{font-family:'Material Icons';font-weight:normal;font-style:normal;font-size:24px;line-height:1;letter-spacing:normal;text-transform:none;display:inline-block;white-space:nowrap;word-wrap:normal;direction:ltr;font-feature-settings:'liga';}@font-face{font-family:'Material Icons';font-style:normal;font-weight:400;src:url(https://fonts.gstatic.com/s/materialicons/v125/flUhRq6tzZclQEJ-Vdg-IuiaDsNcI

In [ ]:

def affiliated_univ(DOI):
  response = requests.get('https://pubmed.ncbi.nlm.nih.gov/?term='+str(DOI))
  soup = BeautifulSoup(response.content, 'html.parser')
  soup.find_all('li', {"data-affiliation-id": "affiliation-1"})

  for text in soup.find_all('li', {"data-affiliation-id": "affiliation-1"}):
    affiliation=text.text[2:]
    break
  return affiliation

In [ ]:
import requests
from bs4 import BeautifulSoup


response = requests.get('https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0069485')
soup = BeautifulSoup(response.content, 'html.parser')

x=soup.find("meta",{"name":"citation_author_institution"})

print(x['content'])

Department of Biochemistry and Molecular Biology, Bengbu Medical College, Bengbu, Anhui, China


In [ ]:
#better function using both pubmed and plos one
def affiliated_univ(DOI):
  try:
    response = requests.get('https://journals.plos.org/plosone/article/authors?id='+str(DOI))
    soup = BeautifulSoup(response.content, 'html.parser')
    affiliation=soup.find("meta",{"name":"citation_author_institution"})
    return affiliation['content']
  except:
    response = requests.get('https://pubmed.ncbi.nlm.nih.gov/?term='+str(DOI))
    soup = BeautifulSoup(response.content, 'html.parser')
    soup.find_all('li', {"data-affiliation-id": "affiliation-1"})

    for text in soup.find_all('li', {"data-affiliation-id": "affiliation-1"}):
      affiliation=text.text[2:]
      break
    return affiliation

In [ ]:
affiliated_univ('10.1016/j.cyto.2012.05.021')

'Department of Preventive and Social Dentistry, School of Dentistry, Institute of Oral Biology, Kyung Hee University, Seoul, Republic of Korea.'

In [ ]:
#affiliation list has a list of the affiliated universities. contains 'None' if there is no info on that-- keep 
#note: this took 20 munites to run D:
import pandas as pd

df = pd.read_csv("Bik dataset - papers with endpoint reached.tsv", sep='\t', encoding = "ISO-8859-1")
journal_link = df['DOI'][54:58]
affiliation_list=[]

for doi in journal_link:
  print(affiliated_univ(doi))
  if affiliated_univ(doi) is None:
    affiliation_list.append('None')
  else:
    affiliation_list.append(affiliated_univ(doi))
print(affiliation_list)


None
Department of Oral and Molecular Microbiology, Osaka University Graduate School of Dentistry, 1-8 Yamadaoka, Japan. okahashi@dent.osaka-u.ac.jp
None
None
['None', 'Department of Oral and Molecular Microbiology, Osaka University Graduate School of Dentistry, 1-8 Yamadaoka, Japan. okahashi@dent.osaka-u.ac.jp', 'None', 'None']


In [ ]:
len(affiliation_list)

4

In [ ]:
affiliated_univ('10.1371/journal.pone.0069583')

'Department of Neurology, The Agnes Ginges Center for Human Neurogenetics, Hadassah University Hospital, Jerusalem, Israel.'

In [ ]:
df = pd.read_csv("Bik dataset - papers with endpoint reached.tsv", sep='\t',encoding = "ISO-8859-1", nrows=214)
all_authors = df['Authors'].tolist()

In [ ]:
# Finding the highest degree obtained from research gate

degrees = ['phd', 'dphil', 'doctor of philosophy', 'doctor of engineering', 'doctor', 'ph.d', 'ph.d.', 'dba', 'masters', 'ms', 'msc']

def get_highest_degree(author_name):
    link = ''
    query = author_name + ' Research Gate'
    for k in search(query, stop=1):
        if 'https://www.researchgate.net/profile/' in k:
            link = k
            response = requests.get(link)
            soup = BeautifulSoup(response.content, 'html.parser')
            print(soup.find_all('title'))
            if len(soup.find_all('title')) == 0:
                return -1
            listofattributes = soup.find_all('title')[0].getText().split('|')
            for word in listofattributes:
                if word.strip().lower() in degrees:
                    return word.strip()
                if 'phd' in word.strip().lower():
                    return 'PhD'
    return -1

In [ ]:
author_list = [x.split(',')[0] for x in all_authors]
highest_degree = []
for author_name in author_list:
    degree = get_highest_degree(author_name)
    # print((author_name, degree)) 
    highest_degree.append(degree)

('Inka Regine Weingaertner', -1)
('Jessica M. Esparza', -1)
('Sreedevi Avasarala', -1)
('Rounak Nassirpour', -1)
[<title>Please Wait... | Cloudflare</title>]
('Jianuo Liu', -1)
[<title>Please Wait... | Cloudflare</title>]
('Yang Wang', -1)
[<title>Please Wait... | Cloudflare</title>]
('Elizabeth Ngoc Hoa Tran', -1)
[<title>Please Wait... | Cloudflare</title>]
('Fabio Sallustio', -1)
('Jia Ma', -1)
[<title>Please Wait... | Cloudflare</title>]
('Yael Friedman-Levi', -1)
[<title>Please Wait... | Cloudflare</title>]
('Yan Zhang', -1)
('Zhongpu Chen', -1)
[<title>Please Wait... | Cloudflare</title>]
('Akrem Abdulsid', -1)
('Fang Liu', -1)
('Jagannath Misra', -1)
[<title>Please Wait... | Cloudflare</title>]
('Trent Fowler', -1)
[<title>Please Wait... | Cloudflare</title>]
('Mojca Jez', -1)
('Zhenni Zhang', -1)
[<title>Please Wait... | Cloudflare</title>]
('Christophe Broca', -1)
[<title>Please Wait... | Cloudflare</title>]
('Sylwia Flis', -1)
[<title>Please Wait... | Cloudflare</title>]
('Ta

In [ ]:
# Finding the area of highest degree obtained from research gate

def get_degree_area(author_name):
    link = ''
    query = author_name + ' Research Gate'
    for k in search(query, stop=1):
        if 'https://www.researchgate.net/profile/' in k:
            link = k
            response = requests.get(link)
            soup = BeautifulSoup(response.content, 'html.parser')
            if len(soup.find_all('title')) == 0:
                return -1
            listofattributes = soup.find_all('title')[0].getText().split('|')
            for word in listofattributes:
                if 'department of' in word.strip().lower():
                    return word.split('of')[1].strip()
    return -1

In [ ]:
degree_area = []
for author_name in author_list:
    area = get_degree_area(author_name)
    # print((author_name, area))
    degree_area.append(area)

In [ ]:
#Function for lab size just using plos one --> keep
def lab_size(DOI):
    response = requests.get('https://journals.plos.org/plosone/article/authors?id='+str(DOI))
    soup = BeautifulSoup(response.content, 'html.parser')
    affiliation=soup.find_all("meta",{"name":"citation_author_institution"})
    univ_list=[]
    for element in affiliation:
        univ_list.append(element['content'])
    for element in univ_list[1:]:
        if element != univ_list[0]:
            univ_list.remove(element)
    return len(univ_list)

In [ ]:
#both
df = pd.read_csv("Bik dataset - papers with endpoint reached.tsv", sep='\t',encoding = "ISO-8859-1", nrows=214)
journal_link = df['DOI']
lab_size_list=[]

for doi in journal_link:
    try:
        lab_size_list.append(lab_size(doi))
    except:
        lab_size_list.append('N/A')


In [ ]:
#replace all the 0s with total numebr of authors
df = pd.read_csv("Bik dataset - papers with endpoint reached.tsv", sep='\t',encoding = "ISO-8859-1", nrows=214)
all_authors = df['Authors'].tolist()

lab_size_list_revised=[]

for index in range(len(lab_size_list)):
    if lab_size_list[index] != 0:
        lab_size_list_revised.append(lab_size_list[index])
    else:
        author_list = list(all_authors[index].split(','))
        lab_size_list_revised.append(len(author_list))

print(lab_size_list_revised)

NameError: ignored